## 国家专利局数据检索

In [1]:
import requests
import re
import pandas as pd

In [6]:
def getResponse():
    # 登录后才能访问的网页
    url = 'http://pss-system.cnipa.gov.cn/sipopublicsearch/patentsearch/executeSmartSearch1207-executeSmartSearch.shtml'
    # 浏览器登录后得到的cookie
    cookie_str = r'wee_username=Y2hlbmhvbmdsaWFuZzEwMQ%3D%3D; wee_password=Y2hlbmhvbmdsaWFuZzEwMQ%3D%3D; ' \
                 r'_gscu_1645064636=64466672dnnv6r52; avoid_declare=declare_pass; ' \
                 r'JSESSIONID=ZKRKiUkmexcotqtYjmPj2K2InSGvexBfteGpWeR7i3A3zgDMtyal!-586422460!982567663'
    # 把cookie字符串处理成字典，以便接下来使用
    cookies = {}
    for line in cookie_str.split(';'):
        key, value = line.split('=', 1)
        cookies[key] = value
    # 设置请求头
    headers = {
        'Host': 'pss-system.cnipa.gov.cn',
        'Origin': 'http://pss-system.cnipa.gov.cn',
        'X-Requested-With': 'XMLHttpRequest',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36',
        'Referer': 'http://pss-system.cnipa.gov.cn/sipopublicsearch/portal/uiIndex.shtml',
    }
    # 设置表单内容
    search = input("请输入需要查找专利的关键字：")
    data = {
        "searchCondition.searchExp": search,
        "searchCondition.dbId": "VDB",
        "resultPagination.limit": 12,
        "searchCondition.searchType": "Sino_foreign",
        "wee.bizlog.modulelevel": "0200101"
    }
    response = requests.post(url, headers=headers, cookies=cookies, data=data).text

    # 翻页需要的网页
    url2 = 'http://pss-system.cnipa.gov.cn/sipopublicsearch/patentsearch/showSearchResult-startWa.shtml'
    # 浏览器登录后得到的cookie
    cookie_str2 = 'WEE_SID=fbBLCqSIZWRkzzAf9XgtAayOrcgsrYdMn0XpNtyBdmCZodXQKHRw!1487439115!376535830!1564627084424; IS_LOGIN=true; wee_username=Y2hlbmhvbmdsaWFuZzEwMQ%3D%3D; wee_password=Y2hlbmhvbmdsaWFuZzEwMQ%3D%3D; _gscu_1645064636=64466672dnnv6r52; avoid_declare=declare_pass; JSESSIONID=fbBLCqSIZWRkzzAf9XgtAayOrcgsrYdMn0XpNtyBdmCZodXQKHRw!1487439115!376535830'
    # 把cookie字符串处理成字典，以便接下来使用
    cookies2 = {}
    for line in cookie_str2.split(';'):
        key, value = line.split('=', 1)
        cookies2[key] = value
    # 设置表单内容
    for i in range(1, 3):
        i = i * 12
        data2 = {
            "resultPagination.limit": "12",
            "resultPagination.sumLimit": "10",
            "resultPagination.start": str(i),
            "searchCondition.searchExp": "复合文本 = (+" + search + "+)",
            "searchCondition.executableSearchExp":"VDB:(TBI='"+search+"')",
            "searchCondition.literatureSF": "复合文本 = (+" + search + "+)",
        }
        response += requests.post(url2, headers=headers, cookies=cookies2, data=data2).text
    return response

In [7]:
# 获得专利的名称、申请号、申请日等
def getResult(response):
    # 名称
    TIVIEW = re.findall(r'"TIVIEW":"(.*?)",', response)
    # 申请号
    APO = re.findall(r'"APO":"(.*?)",', response)
    # 申请日
    APD = re.findall(r'"APD":"(.*?)",', response)
    # 公开（公告）号
    VID = re.findall(r'"VID":"(.*?)",', response)
    # 公开（公告）日
    PD = re.findall(r'"PD":"(.*?)",', response)
    # IPC分类号
    IC = re.findall(r'"IC":"(.*?)",', response)
    # 申请（专利权）人
    PAVIEW = re.findall(r'"PAVIEW":"(.*?)",', response)
    # 发明人
    INVIEW = re.findall(r'"INVIEW":"(.*?)",', response)
    # 代理人
    AGT = re.findall(r'"AGT":"(.*?)",', response)
    # 代理机构
    AGY = re.findall(r'"AGY":"(.*?)",', response)
    list=pd.DataFrame({"名称":TIVIEW,"申请号":APO,"申请日":APD,"公开（公告）号":VID,"公开（公告）日":PD,
                       "IPC分类号":IC,"申请（专利权）人":PAVIEW,"发明人":INVIEW,"代理人":AGT,"代理机构":AGY})
    return list

In [8]:
if __name__ == '__main__':
    writer = pd.ExcelWriter('E:/1.xlsx')
    getResult(getResponse()).to_excel(writer, 'Sheet1')
    writer.save()
    print("保存成功！")

请输入需要查找专利的关键字：青蒿素
保存成功！
